In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import files

files.upload()

In [ ]:
!ls -lha kaggle.json


In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d akshaypawar7/millions-of-movies

 96% 158M/164M [00:02<00:00, 83.2MB/s]
100% 164M/164M [00:02<00:00, 66.5MB/s]


In [1]:
import pandas as pd 
import datetime
import regex as re
import concurrent.futures
import requests
import math
import zipfile

In [12]:
file_path = "millions-of-movies.zip"
zip_file = zipfile.ZipFile(file_path, 'r')
zip_file.extractall()
zip_file.close()


In [2]:
movie_df = pd.read_csv('D:\CUHK\STAT5106\Group Project\Online Resource\movies.csv')

In [3]:
movie_df.drop(['id','keywords','poster_path','backdrop_path','recommendations'],axis =1 ,inplace=True)

In [4]:
movie_df['release_date'] =pd.to_datetime(movie_df['release_date'])
today = pd.to_datetime(datetime.date.today())
filtered_movie_df = movie_df.query('release_date >= 2018 and release_date <= @today and original_language.isin(["en","zh"]) and status == "Released" and genres.isna() == False').drop_duplicates(['title'])

In [5]:
filtered_movie_df['title_rename'] = filtered_movie_df['title'].astype(str).apply(lambda title: re.sub(r'[^A-Za-z0-9 ]+', '', title).replace(' ','_').lower())
filtered_movie_df.reset_index(drop=True, inplace=True)

In [7]:
test = filtered_movie_df.head()

In [15]:
session = requests.Session()

def fetch_scores(title):
    try:
        url = f'https://rotten-tomatoes-api.ue.r.appspot.com/movie/{title}'
        response = session.get(url)
        response.raise_for_status()
        js = response.json()
        return {
            'name': js['name'],
            'tomatometer': js['tomatometer'],
            'audience_score': js['audience_score'],
            'weighted_score': js['weighted_score'],
            'genres': js['genres'],
            'year': js['year']
        }
    except (KeyError, requests.HTTPError, ValueError):
        return None

batch_size = 1000
num_batches = math.ceil(len(test) / batch_size)

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for i in range(num_batches):
        start_index = i * batch_size
        end_index = (i + 1) * batch_size
        batch_df = test[start_index:end_index]

        results = list(executor.map(fetch_scores, batch_df['title_rename']))

        for result, (_, row) in zip(results, batch_df.iterrows()):
            if result is not None:
                row['movie_name'] = result['name']
                row['tomatometer'] = result['tomatometer']
                row['audience_score'] = result['audience_score']
                row['weighted_score'] = result['weighted_score']
                row['rotten_tomato_genres'] = result['genres']
                row['year'] = result['year']


In [ ]:
session = requests.Session()

def fetch_scores(title):
    try:
        url = f'https://rotten-tomatoes-api.ue.r.appspot.com/movie/{title}'
        response = session.get(url)
        response.raise_for_status()
        js = response.json()
        return js['name'],js['tomatometer'], js['audience_score'],js['weighted_score'],js['genres'],js['year']
    except (KeyError, requests.HTTPError, ValueError):
        return None, None, None, None, None, None

batch_size = 1000
num_batches = math.ceil(len(filtered_movie_df) / batch_size)

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for i in range(num_batches):
        start_index = i * batch_size
        end_index = (i + 1) * batch_size
        batch_df = filtered_movie_df[start_index:end_index]

        results = list(executor.map(fetch_scores, batch_df['title_rename']))

        name,tomatometer_scores, audience_scores,weighted_score,rotten_tomato_genres,year = zip(*results)
        batch_df['movie_name'] = name
        batch_df['tomatometer'] = tomatometer_scores
        batch_df['audience_score'] = audience_scores
        batch_df['weighted_score'] = weighted_score
        batch_df['rotten_tomato_genres'] = rotten_tomato_genres
        batch_df['year'] = year

        batch_df.to_csv(f'batch_{i+1}.csv', index=False)

In [ ]:
batch_dfs = {i: pd.read_csv(f'batch_{i}.csv') for i in range(1, num_batches)}
batch1_df = pd.concat([batch_dfs[i] for i in range(1, num_batches)])

In [ ]:
batch1_df.drop(['id','genres','title_rename','title','year','title'],axis=1,inplace=True)
batch1_df.query('tomatometer.isna() == False & audience_score.isna() == False & rotten_tomato_genres.isna() == False', inplace=True)

In [ ]:
batch1_df.to_csv('movies_with_rotten_tomatoes_v1.csv', index=False)